In [59]:
# https://platform.olimpiada-ai.ro/problems/5

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [60]:
train = pd.read_csv("/kaggle/input/wine-classification/train.csv")
test = pd.read_csv("/kaggle/input/wine-classification/test.csv")

len(train), len(test)

(142, 36)

In [61]:
train

,SampleID,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,37,13.28,1.64,2.84,15.5,110.0,2.60,2.68,0.34,1.36,4.60,1.09,2.78,880.0,0
1,31,13.73,1.50,2.70,22.5,101.0,3.00,3.25,0.29,2.38,5.70,1.19,2.71,1285.0,0
2,27,13.39,1.77,2.62,16.1,93.0,2.85,2.94,0.34,1.45,4.80,0.92,3.22,1195.0,0
3,13,13.75,1.73,2.41,16.0,89.0,2.60,2.76,0.29,1.81,5.60,1.15,2.90,1320.0,0
4,149,13.32,3.24,2.38,21.5,92.0,1.93,0.76,0.45,1.25,8.42,0.55,1.62,650.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,143,13.52,3.17,2.72,23.5,97.0,1.55,0.52,0.50,0.55,4.35,0.89,2.06,520.0,2
138,88,11.65,1.67,2.62,26.0,88.0,1.92,1.61,0.40,1.34,2.60,1.36,3.21,562.0,1
139,35,13.51,1.80,2.65,19.0,110.0,2.35,2.53,0.29,1.54,4.20,1.10,2.87,1095.0,0
140,164,12.96,3.45,2.35,18.5,106.0,1.39,0.70,0.40,0.94,5.28,0.68,1.75,675.0,2


In [62]:
def process_df(df):
    # cols = [c for c in df.columns if c not in ['SampleID', 'target']]
    # for col in cols:
    #     df[f'{col}_sqrt'] = np.sqrt(df[col])
    #     df[f'{col}_log'] = np.log10(df[col]*100+1)
    return df

train = process_df(train)
test = process_df(test)

In [63]:
from sklearn.model_selection import train_test_split
from catboost import Pool

features = [c for c in train.columns if c not in ['SampleID', 'target']]
X, y = train[features], train['target']
X_test = test[features]

X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      random_state=42,
                                                      test_size=0.2,
                                                      stratify=y)

train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

In [64]:
# from catboost import CatBoostClassifier

# params = {
#     'iterations': 10,
#     'loss_function': 'MultiClass',
#     'eval_metric': 'TotalF1:average=Macro',
#     'metric_period': 1,
#     'max_depth': 4
# }

# model = CatBoostClassifier(**params)

# model.fit(train_pool ,eval_set=valid_pool)

In [65]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train, y_train)

RandomForestClassifier()

In [66]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_valid).flatten()

score = f1_score(y_valid, y_pred, average='macro')

print(f"Score: {score:5f}")

Score: 1.000000


In [67]:
y_pred = model.predict(X_test).flatten()

subm = pd.DataFrame({
    'SampleID': test['SampleID'],
    'label': y_pred
})

subm.head()

,SampleID,label
0,11,0
1,135,2
2,29,0
3,122,1
4,63,1


In [68]:
subm.to_csv("submission.csv", index=False)